In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision import datasets,models

import matplotlib.pyplot as plt
import os
from PIL import Image
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

In [2]:
path='/home/amirtesh/Downloads/Computers/Python/MachineLearning/Udemy Pytorch/Convolutional neural networks/Felines'

In [3]:
train_transform=transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(0.5),
    transforms.ToTensor()
])

test_transform=transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor()
])

val_transform=transforms.Compose([
    transforms.Resize(size=(224,224)),
    transforms.ToTensor()
])

In [4]:
train_data=datasets.ImageFolder(os.path.join(path,'train'),transform=train_transform)
test_data=datasets.ImageFolder(os.path.join(path,'test'),transform=test_transform)
val_data=datasets.ImageFolder(os.path.join(path,'valid'),transform=val_transform)

In [5]:
len(train_data),len(test_data),len(val_data)

(1174, 25, 25)

In [6]:
class_names=train_data.classes
class_names

['AFRICAN LEOPARD', 'CHEETAH', 'JAGUAR', 'LIONS', 'TIGER']

In [7]:
test_data.classes,val_data.classes

(['AFRICAN LEOPARD', 'CHEETAH', 'JAGUAR', 'LIONS', 'TIGER'],
 ['AFRICAN LEOPARD', 'CHEETAH', 'JAGUAR', 'LIONS', 'TIGER'])

In [8]:
len(class_names)

5

In [9]:
def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer):
    model.train()
    train_loss, train_acc = 0, 0
    
    for batch, (X, y) in enumerate(dataloader):
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module):
    model.eval() 
    test_loss, test_acc = 0, 0
    
    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            test_pred_logits = model(X)
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))
            
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

from tqdm.auto import tqdm

def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn.Module,
          epochs:int):
    
    results={'train_loss':[],
             'train_acc':[],
             'test_loss':[],
             'test_acc':[]}
    
    for i in tqdm(range(epochs)):
        train_loss,train_acc=train_step(model=model,
                                        dataloader=train_dataloader,
                                        loss_fn=loss_fn,
                                        optimizer=optimizer)
        test_loss,test_acc=test_step(model=model,
                                     dataloader=test_dataloader,
                                     loss_fn=loss_fn)
        print(f'Epoch {i} | Train loss: {train_loss:.4f},Train acc: {train_acc:.4f} | Test loss: {test_loss:.4f},Test acc: {test_acc:.4f}')
        results['train_loss'].append(train_loss)
        results['train_acc'].append(train_acc)
        results['test_loss'].append(test_loss)
        results['test_acc'].append(test_acc)

    return results

In [10]:
torch.manual_seed(101)
train_loader=DataLoader(dataset=train_data,batch_size=32,shuffle=True)
test_loader=DataLoader(dataset=test_data,batch_size=1,shuffle=False)
val_loader=DataLoader(dataset=val_data,batch_size=1,shuffle=False)

In [21]:
model1=models.mobilenet_v3_large(pretrained=True)

In [22]:
model1.classifier[3]=nn.Linear(model1.classifier[3].in_features,len(class_names))

In [23]:
torch.manual_seed(101)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model1.parameters(),lr=0.001)

In [29]:
torch.manual_seed(42)

n=1 #11
model1_results=train(
    model=model1,
    train_dataloader=train_loader,
    test_dataloader=test_loader,
    optimizer=optimizer,
    loss_fn=criterion,
    epochs=n
)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 0 | Train loss: 0.0456,Train acc: 0.9882 | Test loss: 0.0092,Test acc: 1.0000


In [30]:
index=list(range(25))

c=0
for i in index:
    model1.eval()
    with torch.no_grad():
        pred=model1(test_data[i][0].view(1,3,224,224)).argmax()

    
    if class_names[test_data[i][1]]==class_names[pred.item()]:
        c+=1

print(f'Number of correct: {c}/{len(test_data)} | Accuracy: {c*100/len(test_data)}')

Number of correct: 25/25 | Accuracy: 100.0


In [31]:
index=list(range(25))

c=0
for i in index:
    model1.eval()
    with torch.no_grad():
        pred=model1(val_data[i][0].view(1,3,224,224)).argmax()

    
    if class_names[val_data[i][1]]==class_names[pred.item()]:
        c+=1

print(f'Number of correct: {c}/{len(val_data)} | Accuracy: {c*100/len(val_data)}')

Number of correct: 24/25 | Accuracy: 96.0
